# Домашнее задание № 7

## Задание 1 (4 балла) 

Обучите 2 модели похожую по архитектуре на модель из ULMFit для задачи классификации текста (датасет - lenta_40k )
В моделях должно быть как минимум два рекуррентных слоя, а финальный вектор для классификации составляться из последнего состояния RNN (так делалось в семинаре), а также AveragePooling и MaxPooling из всех векторов последовательности (конкатенируйте последнее состояния и результаты пулинга). В первой модели используйте обычные слои, а во второй Bidirectional. Рассчитайте по классовую точность/полноту/f-меру для каждой из модели (результаты не должны быть совсем близкие к нулю после обучения на хотя бы нескольких эпохах). 

In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: mps


In [2]:
def preprocess(text: str):
    return [t.strip(".,!?;:'\"()[]{}").lower() for t in text.split()]

DATA_PATH = "lenta_40k.csv.zip"
assert os.path.exists(DATA_PATH), "Файл lenta_40k.csv.zip не найден"

data = pd.read_csv(DATA_PATH)

vocab = Counter()
for t in data.text:  # build vocabulary
    vocab.update(preprocess(t))

word2id = {"PAD": 0, "UNK": 1}
word2id.update({w: i + len(word2id) for i, (w, c) in enumerate(sorted(vocab.items())) if c > 30})

id2word = {i: w for w, i in word2id.items()}
MAX_LEN = int(np.median([len(preprocess(t)) for t in data.text]) + 30)


def encode_texts(texts):
    X = np.zeros((len(texts), MAX_LEN), dtype=np.int64)
    for i, t in enumerate(texts):
        ids = [word2id.get(tok, 1) for tok in preprocess(t)][:MAX_LEN]
        X[i, : len(ids)] = ids
    return X


X = encode_texts(data.text)
label2id = {l: i for i, l in enumerate(sorted(data.topic.unique()))}
id2label = {i: l for l, i in label2id.items()}
y = data.topic.map(label2id).to_numpy(dtype=np.int64)

In [3]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.05, stratify=y, random_state=42
)


class TextDS(Dataset):
    def __init__(self, X, y):
        self.X, self.y = torch.tensor(X), torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx].long(), self.y[idx].long()


def make_loader(X, y, batch_size=256, shuffle=False):
    return DataLoader(TextDS(X, y), batch_size=batch_size, shuffle=shuffle)


train_dl = make_loader(X_train, y_train, shuffle=True)
val_dl = make_loader(X_val, y_val)

In [4]:
class ULMFit(nn.Module):
    """2×LSTM (+BiLSTM option) → concat(last, avg‑pool, max‑pool)"""

    def __init__(self, vocab, emb_dim, hid, n_cls, bidir=False):
        super().__init__()
        self.bidir = bidir
        self.embed = nn.Embedding(vocab, emb_dim, padding_idx=0)
        self.rnn1 = nn.LSTM(emb_dim, hid, batch_first=True, bidirectional=bidir)
        self.rnn2 = nn.LSTM(hid * (2 if bidir else 1), hid, batch_first=True, bidirectional=bidir)
        out_dim = hid * (2 if bidir else 1)
        self.fc = nn.Linear(out_dim * 3, n_cls)

    def forward(self, x):
        e = self.embed(x)
        o1, _ = self.rnn1(e)
        o2, (h2, _) = self.rnn2(o1)

        if self.bidir:
            last = torch.cat([h2[-2], h2[-1]], dim=1)  # (B, 2H)
        else:
            last = h2[-1]  # (B, H)

        avg_pool = o2.mean(1)
        max_pool, _ = o2.max(1)
        return self.fc(torch.cat([last, avg_pool, max_pool], 1))


In [5]:
def print_cls_report(y_true, y_pred, label_map):
    labels = list(range(len(label_map)))    
    target_names = [label_map[i] for i in labels] 
    report = classification_report(
        y_true,
        y_pred,
        labels=labels,
        target_names=target_names,
        digits=3,
        zero_division=0,
    )
    
    print(report)

In [6]:
def train_eval(model: nn.Module, name: str, epochs: int = 5):
    model.to(device)
    opt = Adam(model.parameters(), lr=1e-3)
    crit = nn.CrossEntropyLoss()

    for ep in range(1, epochs + 1):
        model.train()
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            loss = crit(model(xb), yb)
            opt.zero_grad(); loss.backward()
            opt.step()

        model.eval()
        ys, ps = [], []
        with torch.no_grad():
            for xb, yb in val_dl:
                ys.extend(yb.tolist())
                ps.extend(model(xb.to(device)).argmax(1).cpu().tolist())
        if ep%2==0:
            print(f"\n[{name}] Epoch {ep} – per‑class metrics:")
            print_cls_report(ys, ps, id2label)

In [7]:
vanilla = ULMFit(len(word2id), 100, 128, len(label2id), bidir=False)
bidir   = ULMFit(len(word2id), 100, 128, len(label2id), bidir=True)

train_eval(vanilla, "ULMFit‑Vanilla", epochs=10)
train_eval(bidir,  "ULMFit‑BiDir",  epochs=10)


[ULMFit‑Vanilla] Epoch 2 – per‑class metrics:
                   precision    recall  f1-score   support

   69-я параллель      0.000     0.000     0.000         4
       Библиотека      0.000     0.000     0.000         0
           Бизнес      0.000     0.000     0.000        22
      Бывший СССР      0.176     0.019     0.034       159
              Дом      0.000     0.000     0.000        66
         Из жизни      0.000     0.000     0.000        84
   Интернет и СМИ      0.000     0.000     0.000       132
             Крым      0.000     0.000     0.000         2
    Культпросвет       0.000     0.000     0.000         1
         Культура      0.284     0.195     0.231       159
          Легпром      0.000     0.000     0.000         0
              Мир      0.226     0.295     0.256       410
  Наука и техника      0.229     0.237     0.233       160
      Путешествия      0.000     0.000     0.000        21
           Россия      0.262     0.711     0.383       481
Силовые 

## Задание 2 (6 баллов)


На данных википедии (wikiann) обучите и сравните 3 модели:  
1) модель в которой как минимум два рекуррентных слоя, причем один из них GRU, а другой LSTM 
2) модель в которой как минимум 3 рекуррентных слоя идут друг за другом и при этом 2-ой и 3-й слои еще имеют residual connection к изначальным эмбедингам. Для того, чтобы сделать residual connection вам нужно будет использовать одинаковую размерность эмбедингов и количество unit'ов в RNN слоях, чтобы их можно было просуммировать 
3) модель в которой будут и рекуррентные и сверточные слои (как минимум 2 rnn и как минимум 2 cnn слоя). В cnn слоях будьте аккуратны с укорачиванием последовательности и используйте паддинг



Сравните качество по метрикам (точность/полнота/f-мера). Также придумайте несколько сложных примеров и проверьте, какие сущности определяет каждая из моделей.

In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification

wiki = load_dataset("wikiann", "en")
print("WikiANN split sizes:", {k: len(v) for k, v in wiki.items()})

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors="pt")

def tokenize_and_align(batch):
    enc = tokenizer(
        batch["tokens"],
        is_split_into_words=True,
        padding=True,
        truncation=True,
        max_length=128,
        return_attention_mask=True,
    )

    aligned_labels = []
    for i, encoding in enumerate(enc.encodings):
        word_ids = encoding.word_ids
        sent_labels = [batch["ner_tags"][i][wid] if wid is not None else -100 for wid in word_ids]
        aligned_labels.append(sent_labels)

    enc["labels"] = aligned_labels
    return {k: enc[k] for k in ("input_ids", "attention_mask", "labels")}

encoded = wiki.map(tokenize_and_align, batched=True, remove_columns=wiki["train"].column_names)
train_ds, val_ds = encoded["train"], encoded["validation"]

def make_loader_ner(ds, bs=32, shuffle=False):
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, collate_fn=data_collator)

train_ld = make_loader_ner(train_ds, shuffle=True)
val_ld   = make_loader_ner(val_ds)

tags = wiki["train"].features["ner_tags"].feature.names

class NERModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, rnn_cells, use_residual=False, use_cnn=False):
        super().__init__()
        self.use_res, self.use_cnn = use_residual, use_cnn
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnns = nn.ModuleList()
        in_dim = emb_dim
        for cell in rnn_cells:
            self.rnns.append(cell(in_dim, hidden_dim, batch_first=True))
            in_dim = hidden_dim
        if use_cnn:
            self.conv1 = nn.Conv1d(hidden_dim, hidden_dim, 3, padding=1)
            self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, 3, padding=1)
        self.fc = nn.Linear(hidden_dim, len(tags))
    def forward(self, input_ids, attention_mask=None):
        emb = self.embed(input_ids)
        out = emb
        for i, rnn in enumerate(self.rnns):
            out, _ = rnn(out)
            if self.use_res and i > 0:
                out = out + emb
        if self.use_cnn:
            o = F.relu(self.conv1(out.permute(0, 2, 1)))
            out = F.relu(self.conv2(o)).permute(0, 2, 1)
        return self.fc(out)

WikiANN split sizes: {'validation': 10000, 'test': 10000, 'train': 20000}


/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [15]:
from seqeval.metrics import classification_report as ner_classification_report
from tqdm import tqdm
def build_models():
    return [
        ("GRU+LSTM", NERModel(tokenizer.vocab_size, 64, 64, [nn.GRU, nn.LSTM])),
        ("Res3RNN", NERModel(tokenizer.vocab_size, 64, 64, [nn.LSTM]*3, use_residual=True)),
        ("RNN+CNN", NERModel(tokenizer.vocab_size, 64, 64, [nn.LSTM, nn.LSTM], use_cnn=True)),
    ]


def train_ner(model, name: str, epochs: int = 3):
    model.to(device)
    opt = Adam(model.parameters(), lr=2e-3)
    crit = nn.CrossEntropyLoss(ignore_index=-100)

    for ep in range(1, epochs + 1):
        model.train()
        for batch in tqdm(train_ld):
            ids, labels = batch["input_ids"].to(device), batch["labels"].to(device)
            logits = model(ids)                          # (B, L, C)
            loss = crit(logits.view(-1, len(tags)), labels.view(-1))
            opt.zero_grad(); loss.backward(); opt.step()

        model.eval()
        corr = tot = 0
        with torch.no_grad():
            for batch in val_ld:
                ids, labels = batch["input_ids"].to(device), batch["labels"].to(device)
                preds = model(ids).argmax(-1)
                mask = labels != -100
                corr += (preds[mask] == labels[mask]).sum().item()
                tot += mask.sum().item()
        token_acc = corr / tot

        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in tqdm(val_ld):
                ids, labels = batch["input_ids"].to(device), batch["labels"].to(device)
                preds = model(ids).argmax(-1).cpu().tolist()
                true  = labels.cpu().tolist()
                for pred_seq, true_seq in zip(preds, true):
                    seq_preds = []
                    seq_trues = []
                    for p, t in zip(pred_seq, true_seq):
                        if t == -100:
                            continue
                        seq_preds.append(tags[p])
                        seq_trues.append(tags[t])
                    all_preds.append(seq_preds)
                    all_labels.append(seq_trues)

        print(f"\n[{name}] Epoch {ep}")
        print(f"Token-accuracy = {token_acc:.3f}")
        print("Entity-level precision/recall/F1:")
        print(ner_classification_report(all_labels, all_preds, zero_division=0))


In [16]:
models = build_models()
for n, net in models:
        train_ner(net, n, epochs=2)

100%|█████████████████████████████████████████| 313/313 [00:10<00:00, 29.01it/s]



[GRU+LSTM] Epoch 1
Token-accuracy = 0.452
Entity-level precision/recall/F1:
              precision    recall  f1-score   support

         LOC       0.00      0.00      0.00      9726
         ORG       0.00      0.00      0.00      7703
         PER       0.00      0.00      0.00      6919

   micro avg       0.00      0.00      0.00     24348
   macro avg       0.00      0.00      0.00     24348
weighted avg       0.00      0.00      0.00     24348



100%|█████████████████████████████████████████| 313/313 [00:10<00:00, 29.54it/s]



[GRU+LSTM] Epoch 2
Token-accuracy = 0.452
Entity-level precision/recall/F1:
              precision    recall  f1-score   support

         LOC       0.00      0.00      0.00      9726
         ORG       0.00      0.00      0.00      7703
         PER       0.00      0.00      0.00      6919

   micro avg       0.00      0.00      0.00     24348
   macro avg       0.00      0.00      0.00     24348
weighted avg       0.00      0.00      0.00     24348



100%|█████████████████████████████████████████| 313/313 [00:03<00:00, 83.63it/s]


...


[Res3RNN] Epoch 2
Token-accuracy = 0.713
Entity-level precision/recall/F1:
              precision    recall  f1-score   support

         LOC       0.36      0.40      0.38      9726
         ORG       0.14      0.20      0.17      7703
         PER       0.26      0.32      0.29      6919

   micro avg       0.25      0.31      0.28     24348
   macro avg       0.25      0.31      0.28     24348
weighted avg       0.26      0.31      0.28     24348



100%|████████████████████████████████████████| 313/313 [00:02<00:00, 154.14it/s]



[RNN+CNN] Epoch 1
Token-accuracy = 0.681
Entity-level precision/recall/F1:
              precision    recall  f1-score   support

         LOC       0.32      0.40      0.36      9726
         ORG       0.19      0.16      0.17      7703
         PER       0.21      0.27      0.24      6919

   micro avg       0.26      0.29      0.27     24348
   macro avg       0.24      0.28      0.26     24348
weighted avg       0.25      0.29      0.27     24348



100%|████████████████████████████████████████| 313/313 [00:02<00:00, 153.01it/s]



[RNN+CNN] Epoch 2
Token-accuracy = 0.753
Entity-level precision/recall/F1:
              precision    recall  f1-score   support

         LOC       0.52      0.45      0.48      9726
         ORG       0.28      0.29      0.29      7703
         PER       0.37      0.50      0.43      6919

   micro avg       0.39      0.41      0.40     24348
   macro avg       0.39      0.41      0.40     24348
weighted avg       0.40      0.41      0.40     24348



In [17]:
samples = [
    "Apple рассматривает покупку стартапа в области искусственного интеллекта",
    "В понедельник Москва и Питер встретились на стадионе",
]
for text in samples:
    tokens = preprocess(text)
    ids = torch.tensor(encode_texts([text])).to(device)
    for name, model in models:
        pred = model(ids).argmax(-1)[0].tolist()
        print(name, list(zip(tokens, [tags[p] for p in pred[:len(tokens)]])))

GRU+LSTM [('apple', 'O'), ('рассматривает', 'O'), ('покупку', 'O'), ('стартапа', 'O'), ('в', 'O'), ('области', 'O'), ('искусственного', 'O'), ('интеллекта', 'O')]
Res3RNN [('apple', 'O'), ('рассматривает', 'O'), ('покупку', 'I-PER'), ('стартапа', 'O'), ('в', 'O'), ('области', 'I-ORG'), ('искусственного', 'I-ORG'), ('интеллекта', 'O')]
RNN+CNN [('apple', 'I-ORG'), ('рассматривает', 'I-ORG'), ('покупку', 'I-ORG'), ('стартапа', 'I-ORG'), ('в', 'I-ORG'), ('области', 'I-ORG'), ('искусственного', 'I-ORG'), ('интеллекта', 'I-ORG')]
GRU+LSTM [('в', 'O'), ('понедельник', 'O'), ('москва', 'O'), ('и', 'O'), ('питер', 'O'), ('встретились', 'O'), ('на', 'O'), ('стадионе', 'O')]
Res3RNN [('в', 'I-ORG'), ('понедельник', 'I-ORG'), ('москва', 'I-ORG'), ('и', 'I-ORG'), ('питер', 'I-ORG'), ('встретились', 'I-ORG'), ('на', 'I-ORG'), ('стадионе', 'I-ORG')]
RNN+CNN [('в', 'I-ORG'), ('понедельник', 'I-ORG'), ('москва', 'I-ORG'), ('и', 'I-ORG'), ('питер', 'I-ORG'), ('встретились', 'I-ORG'), ('на', 'I-ORG'), (